<a href="https://colab.research.google.com/github/OlekanmaVictoria/Data-science-Portfolio/blob/main/Task_Dental_Image_Analysis_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project: Dental Image Processing Pipeline**
## **Overview**
This pipeline processes a set of images using techniques including image alignment, focus stacking, segmentation, and region measurement. The goal is to create a focus-composited image, segment the image, and measure and annotate different regions within it.

# **1. Downloading Images**
## **Objective:**
 Download a folder of images from Google Drive for processing.

In [ ]:
import gdown

# URL to the Google Drive folder containing images
url = 'https://drive.google.com/drive/folders/1KuUPqc-Gd1YwULdlSkIw9GCFYsHx4izp?usp=sharing'

# Download the folder containing images
gdown.download_folder(url, quiet=False)


Retrieving folder contents


Processing file 1ZWy_k7frsw_YWZlmohP2d_qWbFD-3wJZ 1球あり.png
Processing file 1z4P6M293XPXP6iG6vQPfhbXUyLPECYQi 2.png
Processing file 1BGJGd9-vMgdi0mkgS-IfuAWQO9IEyZYJ 3.png
Processing file 1kNYdha4lu82_Nu36vT8Hrl7EXdebWgFQ 4.png
Processing file 1eSM5TMuJ3EJSpApZDIvMoldTS0jSNEXn 5.png


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1ZWy_k7frsw_YWZlmohP2d_qWbFD-3wJZ
To: /content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/1球あり.png
100%|██████████| 1.91M/1.91M [00:00<00:00, 35.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1z4P6M293XPXP6iG6vQPfhbXUyLPECYQi
To: /content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/2.png
100%|██████████| 1.87M/1.87M [00:00<00:00, 24.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BGJGd9-vMgdi0mkgS-IfuAWQO9IEyZYJ
To: /content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/3.png
100%|██████████| 1.84M/1.84M [00:00<00:00, 28.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kNYdha4lu82_Nu36vT8Hrl7EXdebWgFQ
To: /content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/4.png
100%|██████████| 1.85M/1.85M [00:00<00:00, 40.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eSM5TMuJ3EJSpApZDIvMoldTS0jSNEXn
To: /content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/5.png
100%|██████████| 1.98M/1.98M [

['/content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/1球あり.png',
 '/content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/2.png',
 '/content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/3.png',
 '/content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/4.png',
 '/content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様/5.png']

# **Description:**
The images are downloaded from a shared Google Drive folder using gdown.
Ensure that the URL is correct and accessible.

# 2. **Loading and Aligning Images**
## **Objective:**
Load images and align them to a reference image using feature matching.

In [ ]:
import cv2
import numpy as np
import os

# Path to the folder containing images
folder_path = '/content/大阪歯科大学附属病院有歯補綴咬合学講座堀 圭佑様'

# List all image files
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]

# Load images in color
base_image = cv2.imread(image_files[0])
images = [cv2.imread(img_path) for img_path in image_files]

# Function to align images
def align_images(base_image, images):
    aligned_images = []
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(cv2.cvtColor(base_image, cv2.COLOR_BGR2GRAY), None)

    for img in images:
        keypoints2, descriptors2 = orb.detectAndCompute(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), None)
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        matches = matcher.match(descriptors1, descriptors2)
        matches = sorted(matches, key=lambda x: x.distance)
        src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        matrix, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
        height, width, channels = base_image.shape
        aligned_image = cv2.warpPerspective(img, matrix, (width, height))
        aligned_images.append(aligned_image)
    return aligned_images

# Align the images
aligned_images = align_images(base_image, images)


# **Description:**
Images are aligned to the base image using ORB feature matching and homography transformation. This ensures that all images are registered in the same coordinate system.

# **3. Focus Stacking**
Objective: Create a focus-composited image by combining the aligned images.

In [ ]:
import numpy as np

# Function to focus stack images
def focus_stack(aligned_images):
    stack = np.max(np.array(aligned_images), axis=0)
    return stack

# Create focus-composited image
focused_stack = focus_stack(aligned_images)
cv2.imwrite('Focus_Composited_Image.jpg', focused_stack)


True

# **Description:**
The focus-composited image is generated by taking the maximum pixel values from the aligned images, effectively combining focused regions from multiple images.

# **4. Image Segmentation**
Objective: Segment the focus-composited image to identify different regions.

In [ ]:
from skimage import measure, filters, morphology, color

# Function to segment the image
def segment_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = filters.threshold_otsu(gray_image)
    binary_mask = gray_image > thresh
    cleaned_mask = morphology.remove_small_objects(binary_mask, min_size=500)
    cleaned_mask = morphology.closing(cleaned_mask, morphology.square(3))
    labeled_image = measure.label(cleaned_mask)
    return labeled_image

# Segment the focus-composited image
segmented_image = segment_image(focused_stack)
segmented_image_colored = color.label2rgb(segmented_image, bg_label=0, image=focused_stack)
cv2.imwrite('Segmented_Image.jpg', cv2.cvtColor((segmented_image_colored * 255).astype(np.uint8), cv2.COLOR_RGB2BGR))


True

# **Description:**
The image is segmented using Otsu’s thresholding and morphological operations. This process generates a labeled image where each region corresponds to a different segment.

# **5. Merging Segmented Images**
Objective: Merge segmented regions into a single image.

In [ ]:
# Function to merge segmented images
def merge_segmented_images(image, segments):
    merged_image = np.zeros_like(image)
    for segment in np.unique(segments):
        if segment == 0:
            continue
        mask = segments == segment
        merged_image[mask] = image[mask]
    return merged_image

# Create merged segmented image
merged_image = merge_segmented_images(focused_stack, segmented_image)
cv2.imwrite('Merged_Segmented_Image.jpg', merged_image)


True

# **Description:**
Segmented regions are merged into a single image where each region is highlighted. This step helps in visualizing and analyzing specific areas of interest.

# **6. Annotating and Measuring Regions**
Objective: Measure regions and annotate the segmented image with their labels and areas.

In [ ]:
import pandas as pd

# Function to measure regions and annotate the image
def measure_regions(image, segments):
    regions = measure.regionprops(segments)
    areas = []
    annotated_image = image.copy()
    for idx, region in enumerate(regions):
        if region.label != 0:
            areas.append((region.label, region.area))
            # Draw the label number on the image
            minr, minc, maxr, maxc = region.bbox
            cv2.putText(annotated_image, str(idx + 1), (minc, minr), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return areas, annotated_image

# Annotate segmented image with labels
areas, annotated_image = measure_regions(focused_stack, segmented_image)
cv2.imwrite('Annotated_Segmented_Image.jpg', annotated_image)

# Save areas to Excel with labels
areas_df = pd.DataFrame(areas, columns=['Segment Number', 'Area'])
areas_df.to_excel('Segment_Areas_Report.xlsx', index=False)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Description:**
Each segmented region is measured for its area and annotated with its label on the image. The results are saved in an Excel file for further analysis.

# **Summary of Changes**
**Downloading Images**:The images are downloaded from a Google Drive folder.

**Aligning Images**:
Images are aligned using ORB feature matching and homography transformation.

**Focus Stacking**:
A focus-composited image is created by taking the maximum pixel values from the aligned images.

**Segmenting Image**:
 The focus-composited image is segmented using Otsu’s thresholding and morphological operations.

**Merging Segmented Images**: The segmented image is merged to highlight target areas.

**Annotating Segmented Image**: Each segmented area is labeled with a number corresponding to its measurement.
**Saving to Excel:** The areas of the segmented regions are saved to an Excel file.